In [123]:
import pandas as pd, numpy as np

In [124]:
dataset = pd.read_csv('data_cleaned.csv', index_col = 'InvoiceDate')
dataset.index = pd.to_datetime(dataset.index, format = '%Y-%m-%d %H:%M')

In [125]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532820 entries, 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   InvoiceNo     532820 non-null  object 
 1   StockCode     532820 non-null  object 
 2   Description   532820 non-null  object 
 3   Quantity      532820 non-null  int64  
 4   UnitPrice     532820 non-null  float64
 5   CustomerID    403890 non-null  float64
 6   Country       532820 non-null  object 
 7   FinalPrice    532820 non-null  float64
 8   InvoiceMonth  532820 non-null  object 
 9   Day of week   532820 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 44.7+ MB


In [126]:
print("The number of unique products in this dataset based on description")
dataset['Description'].nunique()

The number of unique products in this dataset based on description


3976

In [127]:
print("The number of unique purchased products in this dataset")
print("Excluding returns aka transactions with quantity negative")
dataset.loc[(dataset['Quantity']>0) ,['Description']].nunique()

The number of unique purchased products in this dataset
Excluding returns aka transactions with quantity negative


Description    3962
dtype: int64

We decided to use only the purchase data and no return data, so we pruned our dataset to include only the positive quantity transactions


In [128]:
transactions = dataset.loc[(dataset['Quantity']>0) ,['InvoiceNo','Description','Quantity']]
transactions.head()

,InvoiceNo,Description,Quantity
InvoiceDate,,,
2010-12-01 08:26:00,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
2010-12-01 08:26:00,536365,WHITE METAL LANTERN,6
2010-12-01 08:26:00,536365,CREAM CUPID HEARTS COAT HANGER,8
2010-12-01 08:26:00,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
2010-12-01 08:26:00,536365,RED WOOLLY HOTTIE WHITE HEART,6


In [129]:
type(transactions)

pandas.core.frame.DataFrame

In [130]:
transactions.shape

(524214, 3)

In [131]:
transactions = transactions.groupby(['InvoiceNo','Description'])['Quantity'].sum()

In [132]:
transactions.head(20)

InvoiceNo  Description                        
536365     CREAM CUPID HEARTS COAT HANGER          8
           GLASS STAR FROSTED T-LIGHT HOLDER       6
           KNITTED UNION FLAG HOT WATER BOTTLE     6
           RED WOOLLY HOTTIE WHITE HEART           6
           SET 7 BABUSHKA NESTING BOXES            2
           WHITE HANGING HEART T-LIGHT HOLDER      6
           WHITE METAL LANTERN                     6
536366     HAND WARMER RED POLKA DOT               6
           HAND WARMER UNION JACK                  6
536367     ASSORTED COLOUR BIRD ORNAMENT          32
           BOX OF 6 ASSORTED COLOUR TEASPOONS      6
           BOX OF VINTAGE ALPHABET BLOCKS          2
           BOX OF VINTAGE JIGSAW BLOCKS            3
           DOORMAT NEW ENGLAND                     4
           FELTCRAFT PRINCESS CHARLOTTE DOLL       8
           HOME BUILDING BLOCK WORD                3
           IVORY KNITTED MUG COSY                  6
           LOVE BUILDING BLOCK WORD                3

In [133]:
type(transactions)

pandas.core.series.Series

We decided to convert this Series data into a Dataframe by unstacking it.


In [134]:
transactions = transactions.unstack(fill_value=0)

In [135]:
transactions.head(1)

Description,*USB OFFICE MIRROR BALL,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We have a dataset with each invoice number mapped to the product vector.
Each row of this dataset contains information as to how much quantity of each product was ordered using that invoice
We have the data in the format needed for apriori algorithm

In [136]:
transactions.shape

(19694, 3962)

In [137]:
transactions = (transactions > 0)
transactions.shape

(19694, 3962)

In [138]:
from mlxtend.frequent_patterns import apriori

We decided to have our minimum support threshold to determine frequent item sets as 0.0325.
By default, apriori returns the column indices of the items, which may be useful in downstream operations such as association rule mining. For better readability, we can set use_colnames=True to convert these integer values into the respective item names:

In [139]:
frequent_itemsets = apriori(transactions, min_support= 0.0325, use_colnames=True)

In [140]:
len(frequent_itemsets)

118

In [141]:
frequent_itemsets.tail()

,support,itemsets
113,0.041840,"(JUMBO BAG RED RETROSPOT, JUMBO BAG PINK POLKA..."
114,0.034528,"(JUMBO BAG RED RETROSPOT, JUMBO SHOPPER VINTAG..."
115,0.036762,"(JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SUKI)"
116,0.032548,"(LUNCH BAG RED RETROSPOT, LUNCH BAG BLACK SKULL)"
117,0.033208,"(LUNCH BAG RED RETROSPOT, LUNCH BAG SUKI DESIGN)"


Out of the 118 frequent itemsets we decided to mine some association rules.
Our metric of interest is Confidence -> which would help us get the most predictable rules
The threshold level for confidence metric is 0.7

In [144]:
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.051437,0.054077,0.038946,0.757157,14.001361,0.036164,3.895202
1,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.054077,0.051437,0.038946,0.720188,14.001361,0.036164,3.389999
2,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.061846,0.106022,0.041840,0.676519,6.380921,0.035283,2.763617
3,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.060120,0.106022,0.036762,0.611486,5.767536,0.030388,2.301021
